In [28]:
# import required libraries
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn import metrics, svm
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Uploading Data

In [29]:
sheep = np.load(r"data\full_numpy_bitmap_sheep.npy")
# print(sheep.shape)
giraffe = np.load(r"data\full_numpy_bitmap_giraffe.npy")
# print(giraffe.shape)

sheep = sheep[:5000]
giraffe = giraffe[:5000]

Combining Data Sets and Splitting into Training, Validation, Testing Sets

In [30]:
def combine_data(*args):
    X = np.zeros((0,784))
    Y = np.zeros((0,1))
    for i in range(len(args)):
        X = np.concatenate((X, args[i]))
        Y = np.concatenate((Y, np.ones((args[i].shape[0],1))*i))
    Y = Y.reshape(Y.shape[0])
    return X, Y

def split_data(X, Y, split=(0.8,0.1,0.1)):
    # defaulting to an 80/10/10 train/test/val split
    train, test, val = split
    X_train, X_testval, Y_train, Y_testval = train_test_split(X, Y, test_size=test+val, random_state=42, shuffle=True)
    X_test, X_val, Y_test, Y_val = train_test_split(X_testval, Y_testval, test_size=val/(test+val), random_state=42, shuffle=True)
    return X_train, X_test, X_val, Y_train, Y_test, Y_val

X, Y = combine_data(sheep, giraffe)
X_train, X_test, X_val, Y_train, Y_test, Y_val = split_data(X, Y)

SVM classifier function

In [31]:
def create_classifier(X_train, Y_train, kernel="linear", degree=1, C=1.0): # default values
    classifier = svm.SVC(kernel=kernel, degree=degree, C=C)
    classifier.fit(X_train, Y_train)
    return classifier

Accuracy

In [32]:
def accuracy(classifier, X_val, Y_val):
    return classifier.score(X_val, Y_val)

# def plot_confusion_matrix(classifier, X_val, Y_val):

Test for best normalization using default values

In [33]:
# normalizations
normalizations = [None, StandardScaler(), MinMaxScaler()]

def test_norm(X_train, Y_train, X_val, Y_val, normalizations):
    data = {"Normalization": [], "Accuracy": []}
    for norm in normalizations:
        if norm is not None:
            scaler = norm
            X_train_norm = scaler.fit_transform(X_train)
            X_val_norm = scaler.transform(X_val)
        else:
            X_train_norm = X_train
            X_val_norm = X_val
        classifier = create_classifier(X_train_norm, Y_train)
        acc = accuracy(classifier, X_val_norm, Y_val)
        data["Normalization"].append(norm)
        data["Accuracy"].append(acc)
    return pd.DataFrame(data)

df = test_norm(X_train, Y_train, X_val, Y_val, normalizations)
print(df)

      Normalization  Accuracy
0              None     0.927
1  StandardScaler()     0.942
2    MinMaxScaler()     0.967


Test Different Kernels and Degrees
Using Our Best Normalization Algorithm (Normalization=StandardScaler?? looks like there is no difference)

In [34]:
def test_kern_deg_reg(kernels, degrees, C_vals):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    data = {"Kernel": [], "Degree": [], "C": [], "Accuracy": []}
    for kernel in kernels:
        for degree in degrees:
            for C in C_vals:
                classifier = create_classifier(X_train_scaled, Y_train, kernel=kernel, degree=degree, C=C)
                acc = accuracy(classifier, X_val_scaled, Y_val)
                data["C"].append(C)
                data["Kernel"].append(kernel)
                data["Degree"].append(degree)
                data["Accuracy"].append(acc)
    df = pd.DataFrame(data)
    return df

C_vals = [0.01, 0.1, 1, 10]
kernels = ["linear", "poly", "rbf"]
degrees = [1, 2, 3, 4, 5]
df = test_kern_deg_reg(kernels, degrees, C_vals)
print(df)

In [ ]:
# def test_kern_deg_reg_mm(kernels, degrees, C_vals):
#     scaler = MinMaxScaler()
#     X_train_scaled = scaler.fit_transform(X_train)
#     X_val_scaled = scaler.transform(X_val)
#     data = {"Kernel": [], "Degree": [], "C": [], "Accuracy": []}
#     for kernel in kernels:
#         for degree in degrees:
#             for C in C_vals:
#                 classifier = create_classifier(X_train_scaled, Y_train, kernel=kernel, degree=degree, C=C)
#                 acc = accuracy(classifier, X_val_scaled, Y_val)
#                 data["C"].append(C)
#                 data["Kernel"].append(kernel)
#                 data["Degree"].append(degree)
#                 data["Accuracy"].append(acc)
#     df = pd.DataFrame(data)
#     return df

# C_vals = [0.01, 0.1, 1, 10]
# kernels = ["linear", "poly", "rbf"]
# degrees = [1, 2, 3, 4, 5]
# df = test_kern_deg_reg_mm(kernels, degrees, C_vals)
# print(df)

    Kernel  Degree      C  Accuracy
0   linear       1   0.01     0.965
1   linear       1   0.10     0.960
2   linear       1   1.00     0.960
3   linear       1  10.00     0.960
4   linear       2   0.01     0.965
5   linear       2   0.10     0.960
6   linear       2   1.00     0.960
7   linear       2  10.00     0.960
8   linear       3   0.01     0.965
9   linear       3   0.10     0.960
10  linear       3   1.00     0.960
11  linear       3  10.00     0.960
12  linear       4   0.01     0.965
13  linear       4   0.10     0.960
14  linear       4   1.00     0.960
15  linear       4  10.00     0.960
16  linear       5   0.01     0.965
17  linear       5   0.10     0.960
18  linear       5   1.00     0.960
19  linear       5  10.00     0.960
20    poly       1   0.01     0.955
21    poly       1   0.10     0.960
22    poly       1   1.00     0.960
23    poly       1  10.00     0.955
24    poly       2   0.01     0.950
25    poly       2   0.10     0.960
26    poly       2   1.00   

In [ ]:
# def test_kern_deg_reg_none(kernels, degrees, C_vals):
#     data = {"Kernel": [], "Degree": [], "C": [], "Accuracy": []}
#     for kernel in kernels:
#         for degree in degrees:
#             for C in C_vals:
#                 classifier = create_classifier(X_train, Y_train, kernel=kernel, degree=degree, C=C)
#                 acc = accuracy(classifier, X_val, Y_val)
#                 data["C"].append(C)
#                 data["Kernel"].append(kernel)
#                 data["Degree"].append(degree)
#                 data["Accuracy"].append(acc)
#     df = pd.DataFrame(data)
#     return df

# C_vals = [0.01, 0.1, 1, 10]
# kernels = ["linear", "poly", "rbf"]
# degrees = [1, 2, 3, 4, 5]
# df = test_kern_deg_reg_none(kernels, degrees, C_vals)
# print(df)

    Kernel  Degree      C  Accuracy
0   linear       1   0.01     0.960
1   linear       1   0.10     0.960
2   linear       1   1.00     0.960
3   linear       1  10.00     0.960
4   linear       2   0.01     0.960
5   linear       2   0.10     0.960
6   linear       2   1.00     0.960
7   linear       2  10.00     0.960
8   linear       3   0.01     0.960
9   linear       3   0.10     0.960
10  linear       3   1.00     0.960
11  linear       3  10.00     0.960
12  linear       4   0.01     0.960
13  linear       4   0.10     0.960
14  linear       4   1.00     0.960
15  linear       4  10.00     0.960
16  linear       5   0.01     0.960
17  linear       5   0.10     0.960
18  linear       5   1.00     0.960
19  linear       5  10.00     0.960
20    poly       1   0.01     0.955
21    poly       1   0.10     0.960
22    poly       1   1.00     0.960
23    poly       1  10.00     0.955
24    poly       2   0.01     0.945
25    poly       2   0.10     0.960
26    poly       2   1.00   